In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import keras.layers as lay
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

: 

In [ ]:
class fade_in(keras.Layer):
    def __init__(self, alpha=0.0, activity_regularizer=None, trainable=True, dtype=None, autocast=True, name=None, **kwargs):
        super().__init__(activity_regularizer=activity_regularizer, trainable=trainable, dtype=dtype, autocast=autocast, name=name, **kwargs)
        self.alpha = tf.Variable(alpha,trainable=False)

    def call(self,inputs):
        antigo, novo = inputs
        


def g_block(x, filters):
    x = lay.UpSampling2D((2,2))(x)
    x = lay.Conv2D(filters,(3,3),(1,1),'same')(x)
    x = lay.LeakyReLU(.2)(x)
    return x


def build_generator_4x4(latent_dim=128):
    inputs = lay.Input((latent_dim))
    x = inputs
    x = lay.Dense(4*4*512)(x)
    x = lay.Reshape((4,4,512))(x)
    img_output = lay.Conv2D(3,(1,1),(1,1),'same',activation='sigmoid')(x)
    return keras.Model(inputs,img_output)
